# HW3: Part I
Completing the https://www.dataquest.io/blog/web-scraping-tutorial-python/ tutorial

In [1]:
import requests

page = requests.get("http://dataquestio.github.io/web-scraping-pages/simple.html")
page

<Response [200]>

In [2]:
page.status_code

200

In [3]:
page.content

b'<!DOCTYPE html>\n<html>\n    <head>\n        <title>A simple example page</title>\n    </head>\n    <body>\n        <p>Here is some simple content for this page.</p>\n    </body>\n</html>'

### Parsing a page with BeautifulSoup

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   A simple example page
  </title>
 </head>
 <body>
  <p>
   Here is some simple content for this page.
  </p>
 </body>
</html>


In [5]:
list(soup.children)

['html', '\n', <html>
 <head>
 <title>A simple example page</title>
 </head>
 <body>
 <p>Here is some simple content for this page.</p>
 </body>
 </html>]

In [6]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.NavigableString, bs4.element.Tag]

In [7]:
html = list(soup.children)[2]
list(html.children)

['\n', <head>
 <title>A simple example page</title>
 </head>, '\n', <body>
 <p>Here is some simple content for this page.</p>
 </body>, '\n']

In [8]:
body = list(html.children)[3]
list(body.children)

['\n', <p>Here is some simple content for this page.</p>, '\n']

In [9]:
p = list(body.children)[1]
p.get_text()

'Here is some simple content for this page.'

### Finding all instances of a tag at once

In [10]:
soup = BeautifulSoup(page.content, 'html.parser')
soup.find_all('p')

[<p>Here is some simple content for this page.</p>]

In [11]:
soup.find_all('p')[0].get_text()

'Here is some simple content for this page.'

In [12]:
soup.find('p')

<p>Here is some simple content for this page.</p>

### Searching for tags by class and id

In [13]:
page = requests.get("http://dataquestio.github.io/web-scraping-pages/ids_and_classes.html")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<html>
<head>
<title>A simple example page</title>
</head>
<body>
<div>
<p class="inner-text first-item" id="first">
                First paragraph.
            </p>
<p class="inner-text">
                Second paragraph.
            </p>
</div>
<p class="outer-text first-item" id="second">
<b>
                First outer paragraph.
            </b>
</p>
<p class="outer-text">
<b>
                Second outer paragraph.
            </b>
</p>
</body>
</html>

In [14]:
soup.find_all('p', class_='outer-text')

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [15]:
soup.find_all(class_="outer-text")

[<p class="outer-text first-item" id="second">
 <b>
                 First outer paragraph.
             </b>
 </p>, <p class="outer-text">
 <b>
                 Second outer paragraph.
             </b>
 </p>]

In [16]:
# To search by element
soup.find_all(id="first")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>]

### Using CSS Selectors

In [17]:
soup.select("div p")

[<p class="inner-text first-item" id="first">
                 First paragraph.
             </p>, <p class="inner-text">
                 Second paragraph.
             </p>]

# Downloading weather data

In [18]:
page = requests.get("http://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="seven-day-forecast")
forecast_items = seven_day.find_all(class_="tombstone-container")
tonight = forecast_items[0]
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Tonight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Tonight: Mostly clear, with a low around 44. North northwest wind 5 to 7 mph. " class="forecast-icon" src="newimages/medium/nfew.png" title="Tonight: Mostly clear, with a low around 44. North northwest wind 5 to 7 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Clear
 </p>
 <p class="temp temp-low">
  Low: 44 °F
 </p>
</div>


## Question 2:  How many elements are in forecast_items?  Does this make sense?  Explain.

In [21]:
len(forecast_items)

9

There are 9 elements in forecast_items.  Yes, this makes sense because there are forecast items for tonight, and the next four days (day and night) for a total of 9 forecast items.

### Extracting information from the page

In [22]:
period = tonight.find(class_="period-name").get_text()
short_desc = tonight.find(class_="short-desc").get_text()
temp = tonight.find(class_="temp").get_text()

print(period)
print(short_desc)
print(temp)

Tonight
Mostly Clear
Low: 44 °F


In [23]:
img = tonight.find("img")
desc = img['title']

print(desc)

Tonight: Mostly clear, with a low around 44. North northwest wind 5 to 7 mph. 


### Extracting all the information from the page

In [24]:
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Tonight',
 'Thursday',
 'ThursdayNight',
 'Friday',
 'FridayNight',
 'Saturday',
 'SaturdayNight',
 'Sunday',
 'SundayNight']

In [25]:
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Mostly Clear', 'Mostly Sunny', 'Mostly Cloudy', 'Partly Sunnythen ChanceRain', 'Rain', 'Rain andBreezy', 'ShowersLikely andBreezy', 'ChanceShowers', 'Slight ChanceRain']
['Low: 44 °F', 'High: 55 °F', 'Low: 46 °F', 'High: 56 °F', 'Low: 50 °F', 'High: 57 °F', 'Low: 51 °F', 'High: 57 °F', 'Low: 50 °F']
['Tonight: Mostly clear, with a low around 44. North northwest wind 5 to 7 mph. ', 'Thursday: Mostly sunny, with a high near 55. North wind around 7 mph. ', 'Thursday Night: Mostly cloudy, with a low around 46. Calm wind becoming southwest around 6 mph after midnight. ', 'Friday: A 30 percent chance of rain after 4pm.  Mostly cloudy, with a high near 56. Light southwest wind becoming south southwest 5 to 10 mph in the morning. ', 'Friday Night: Rain, mainly after 10pm.  Low around 50. Southeast wind 11 to 13 mph.  Chance of precipitation is 80%. New precipitation amounts between a tenth and quarter of an inch possible. ', 'Saturday: Rain before 10am, then showers likely after 10am.  High 

### Combining our data into a Pandas Dataframe

In [26]:
import pandas as pd
weather = pd.DataFrame({
        "period": periods, 
        "short_desc": short_descs, 
        "temp": temps, 
        "desc":descs
    })
weather

,desc,period,short_desc,temp
0,"Tonight: Mostly clear, with a low around 44. N...",Tonight,Mostly Clear,Low: 44 °F
1,"Thursday: Mostly sunny, with a high near 55. N...",Thursday,Mostly Sunny,High: 55 °F
2,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Mostly Cloudy,Low: 46 °F
3,Friday: A 30 percent chance of rain after 4pm....,Friday,Partly Sunnythen ChanceRain,High: 56 °F
4,"Friday Night: Rain, mainly after 10pm. Low ar...",FridayNight,Rain,Low: 50 °F
5,"Saturday: Rain before 10am, then showers likel...",Saturday,Rain andBreezy,High: 57 °F
6,Saturday Night: Showers likely. Mostly cloudy...,SaturdayNight,ShowersLikely andBreezy,Low: 51 °F
7,"Sunday: A chance of showers. Mostly cloudy, w...",Sunday,ChanceShowers,High: 57 °F
8,Sunday Night: A slight chance of rain. Mostly...,SundayNight,Slight ChanceRain,Low: 50 °F


In [27]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')
temp_nums

0    44
1    55
2    46
3    56
4    50
5    57
6    51
7    57
8    50
Name: temp_num, dtype: object

In [28]:
weather["temp_num"].mean()

51.77777777777778

In [29]:
is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night
is_night

0     True
1    False
2     True
3    False
4     True
5    False
6     True
7    False
8     True
Name: temp, dtype: bool

In [30]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
0,"Tonight: Mostly clear, with a low around 44. N...",Tonight,Mostly Clear,Low: 44 °F,44,True
2,"Thursday Night: Mostly cloudy, with a low arou...",ThursdayNight,Mostly Cloudy,Low: 46 °F,46,True
4,"Friday Night: Rain, mainly after 10pm. Low ar...",FridayNight,Rain,Low: 50 °F,50,True
6,Saturday Night: Showers likely. Mostly cloudy...,SaturdayNight,ShowersLikely andBreezy,Low: 51 °F,51,True
8,Sunday Night: A slight chance of rain. Mostly...,SundayNight,Slight ChanceRain,Low: 50 °F,50,True


## Question 3:  Provide the code necessary to extract the information shown in the green box below.  Note: your information will look different than the one shown as I took this screenshot several weeks ago.

In [34]:
table = soup.find('table')
table

<table>
<tr>
<td class="text-right"><b>Humidity</b></td>
<td>42%</td>
</tr>
<tr>
<td class="text-right"><b>Wind Speed</b></td>
<td>NA</td>
</tr>
<tr>
<td class="text-right"><b>Barometer</b></td>
<td>NA</td>
</tr>
<tr>
<td class="text-right"><b>Dewpoint</b></td>
<td>29°F (-2°C)</td>
</tr>
<tr>
<td class="text-right"><b>Visibility</b></td>
<td>NA</td>
</tr>
<tr>
<td class="text-right"><b>Last update</b></td>
<td>
                02 Jan 5:43 pm PST             </td>
</tr>
</table>

In [39]:
tds = table.find_all('td')
tds

[<td class="text-right"><b>Humidity</b></td>,
 <td>42%</td>,
 <td class="text-right"><b>Wind Speed</b></td>,
 <td>NA</td>,
 <td class="text-right"><b>Barometer</b></td>,
 <td>NA</td>,
 <td class="text-right"><b>Dewpoint</b></td>,
 <td>29°F (-2°C)</td>,
 <td class="text-right"><b>Visibility</b></td>,
 <td>NA</td>,
 <td class="text-right"><b>Last update</b></td>,
 <td>
                 02 Jan 5:43 pm PST             </td>]

In [53]:
wxDict = {}
for i in range(0,len(tds),2):
    wxDict[tds[i].find('b').get_text()] = tds[i+1].get_text().strip()
wxDict

{'Barometer': 'NA',
 'Dewpoint': '29°F (-2°C)',
 'Humidity': '42%',
 'Last update': '02 Jan 5:43 pm PST',
 'Visibility': 'NA',
 'Wind Speed': 'NA'}